In [1]:
import twint
file = open("./test1.json","w")
file.close()


In [2]:
# clear file
#Twint Search
d = twint.Config()
d.Search = "hash AND software "
# d.Username = "msftsecurity"
d.Limit = 150
# d.Store_csv = True
d.Store_object = True
d.Store_json = True
d.Output = "./test1.json"
# d.Database = "tweets.db"


x = twint.run.Search(d)

1220797800998428673 2020-01-24 11:57:31 Pacific Standard Time <goffspice> - I've just read through the #MCRMK hash tag and seen 2 accounts using the same screenshot. The time and phone network and type of phone software is identical. Be careful. 🖤
1220417721369866241 2020-01-23 10:47:13 Pacific Standard Time <randomtechsolu1> PicoLucid: This is a solution for the problem of non-inclusive pseudonymous software architecting. This will benefit CTOs, and will do so by considering faster audits by installing hash tables in approval flow scenarios.
1220369710530752512 2020-01-23 07:36:27 Pacific Standard Time <abdul_ashkar> I am suggesting to use this amazing software. An easier, safer, and faster way to create, issue, and print checks for business purposes. Online Check Writer delivers the tools we need to manage, document, and track our checks.
1220330815109902337 2020-01-23 05:01:53 Pacific Standard Time <elharo> Autogenerating a few getter and setter methods, maybe an equals method or a 

1216879692411740160 2020-01-13 16:28:21 Pacific Standard Time <TinyGiant1986> What mining software do you use for crypto mining? Along with what’s your current set up and hash rate?
1216639134950924288 2020-01-13 00:32:28 Pacific Standard Time <SecurityMagnate> threatmeter: mruby 2.1.0 hash-ext.c hash_slice memory corruption A vulnerability was found in mruby 2.1.0 (Programming Language Software) and classified as critical. Affected by this issue is the function hash_slice of the file mrbgems/mruby-hash-ext/src…  https://ift.tt/2NcHfJ6 
1216632487155326977 2020-01-13 00:06:03 Pacific Standard Time <threatmeter> mruby 2.1.0 hash-ext.c hash_slice memory corruption A vulnerability was found in mruby 2.1.0 (Programming Language Software) and classified as critical. Affected by this issue is the function hash_slice of the file mrbgems/mruby-hash-ext/src/hash-ext.c. …  https://ift.tt/2NcHfJ6 
1216488870054785024 2020-01-12 14:35:22 Pacific Standard Time <eriktheplaid> Pro tip: If your enterp

1211619299108032514 2019-12-30 04:05:26 Pacific Standard Time <hash_1993> Ada software tak untuk construct simple model/ figures untuk research papers lol
1211558328821665792 2019-12-30 00:03:09 Pacific Standard Time <morodog> #HackingNews #check How, why, and when you should hash check: Here at ProPrivacy we just love open source software. This is mainly because, despite not being perfect, open source provides the only way to know for sure that a program is on the level. One…  http://bit.ly/2F6Nwli  pic.twitter.com/0XIdkfuT1d
1211410650473226241 2019-12-29 14:16:20 Pacific Standard Time <randomtechsolu1> GenAurum: This is a solution for the problem of inaccessible multi-bank customer service. This will benefit customers, and will do so by installing hash tables in user logins for better software architecting.
1211402968991510528 2019-12-29 13:45:49 Pacific Standard Time <LefterisJP> What do you mean? That the most important participants that need to have updated their software is the 

1206883622000693250 2019-12-17 02:27:32 Pacific Standard Time <JeZ_Zc> Ich möchte mal stark davon ausgehen, daß ein Großteil der Großkonzerne & Behörden in Deutschland keine Hash-werte oder überhaupt irgendwas verschlüsselt speichern, oder übermitteln. Klar, modernere Betreiber haben Hash-werte, aber nicht die übliche 15 Jahre veraltete Software.
1206716369791795200 2019-12-16 15:22:56 Pacific Standard Time <IAintShootinMis> So a certain $vendor of security software sent us unsigned agents,  then blamed the hash... weird you compiled the code and published the hash.... wish I worked with @blackroomsec or had a soundboard for vendor calls 🤣
1206601364782092289 2019-12-16 07:45:57 Pacific Standard Time <CryptoBabel> Like trying to say that trusting individually hosted software and/or third party is more vigilance and less risky than trusting in the competition of hash in the system as a whole to maintain the rules. Seems disingenuous to me.
1206545455099854849 2019-12-16 04:03:47 Pacific

1200141935668801541 2019-11-28 11:58:29 Pacific Standard Time <DiabeticCyborgg> Chord: Building a DHT (Distributed Hash Table) in Golang by @arriqaaq  https://link.medium.com/TX2v0z0LF1  #Blockchain #Algorithms #Cryptocurrency #Go #Software
1200041931092512770 2019-11-28 05:21:06 Pacific Standard Time <oscpacey> Anyone wondering how BSV consensus works? Slightly different from Satoshi's original vision in that it uses a custom software licence "Open BSV" which ironically closes use of the software to the blockchain defined by specified hash:   https://github.com/bitcoin-sv/bitcoin-sv/blob/master/LICENSE … pic.twitter.com/EBrncExi32
1199542738733154304 2019-11-26 20:17:29 Pacific Standard Time <toastido> Flight attendant: Is there a doctor onboard?  Dad: *nudging me* that should've been you  Me: Not now Dad  Dad: Not asking for a software developer to help, are they?  Me: Dad, there's a medical emergency happening right now  Dad: Go and see if “use a hash table" helps  https://twitter.c

In [ ]:
# Search terms
# 0-day AND exploit
# “0-day”, “CVE-“, “CVE-2018-*”, “CVE-2019-*”
# usernames:
# kibbsy
# it_securitynews
# msftsecurity

In [ ]:
# x = twint.run.Search(d)
# twint -u noneprivacy --csv --output "/Users/psingh4/harsh/test4.json" --lang en --translate --translate-dest it --limit 100

In [ ]:
# d.__dict__.keys()

In [ ]:
# twint.run.Search(d)
# twint.output

In [14]:
f= open("../Twint Data/4000 rows of data.json","r",errors='ignore')
line1 = f.readlines()
arr = []
for x in line1:
    arr.append(x)
f.close()    

In [15]:
arr.insert(0,"[")
arr.append(']')
for i in range(1,len(arr)-2):
    arr[i] = arr[i][:-1] + "," + arr[i][-1:]

In [16]:
file = open("../test.json","w")
for i in range(len(arr)):
    file.write(arr[i])
file.close()

In [17]:
f= open("../test.json","r",errors='ignore')
line1 = f.readlines()
data = []
for x in line1:
    data.append(x)
f.close()

In [18]:
# Removing non ASCII characters
for i in range(len(data)):
    data[i] = (''.join([i if ord(i) < 128 else ' ' for i in str(data[i])]))

In [19]:
file = open("./data.json","w")
for i in range(len(data)):
    file.write(data[i])
file.close()

In [22]:
import json
with open("./data.json", "r",errors='ignore') as read_file:
    data = json.load(read_file)

In [23]:
import mysql.connector

In [24]:
mydb = mysql.connector.connect(user='admin', password='Private2712!',
                              host='database-1.cok63qqiofsd.us-east-1.rds.amazonaws.com',
                              database='data')

In [25]:
mycursor = mydb.cursor()

In [26]:
sql = """INSERT INTO train_data (created_at, conversation_id, id, date, time, timezone, user_id, username, name, place, tweet, mentions, urls, replies_count, 
        retweets_count, likes_count, hashtags, cashtags, link, retweet, video, near, geo, source, user_rt_id, user_rt, retweet_id,
        retweet_date, translate, trans_src, trans_dest, photos, reply_to) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

val = []
for i in range(len(data)):
    val.append((data[i]['created_at'], data[i]['conversation_id'], str(data[i]['id']), data[i]['date'], data[i]['time'], data[i]['timezone'], data[i]['user_id'], data[i]['username'],
      data[i]['name'], data[i]['place'], data[i]['tweet'], str(data[i]['mentions']), str(data[i]['urls']), data[i]['replies_count'],
      data[i]['retweets_count'], data[i]['likes_count'], str(data[i]['hashtags']), str(data[i]['cashtags']), data[i]['link'], data[i]['retweet'],
      data[i]['video'], data[i]['near'], data[i]['geo'], data[i]['source'], data[i]['user_rt_id'], data[i]['user_rt'],
      data[i]['retweet_id'], data[i]['retweet_date'], data[i]['translate'], data[i]['trans_src'], data[i]['trans_dest'],str(data[i]['photos']), 
       str(data[i]['reply_to'])))
    
mycursor.executemany(sql, val)

mydb.commit()

print(mycursor.rowcount, "record inserted.")

4005 record inserted.


In [27]:
mydb.close()

In [ ]:
val = []
for i in range(len(data)):
    val.append((data[i]['created_at'], data[i]['conversation_id'], data[i]['date'], data[i]['time'], data[i]['timezone'], data[i]['user_id'], data[i]['username'],
      data[i]['name'], data[i]['place'], data[i]['tweet'], str(data[i]['mentions']), str(data[i]['urls']), data[i]['replies_count'],
      data[i]['retweets_count'], data[i]['likes_count'], str(data[i]['hashtags']), str(data[i]['cashtags']), data[i]['link'], data[i]['retweet'],
      data[i]['video'], data[i]['near'], data[i]['geo'], data[i]['source'], data[i]['user_rt_id'], data[i]['user_rt'],
      data[i]['retweet_id'], data[i]['retweet_date'], data[i]['translate'], data[i]['trans_src'], data[i]['trans_dest'],0,str(data[i]['photos']), 
       str(data[i]['reply_to'])))

In [ ]:
# mydb.commit()
# !pip install tensorflow
# print(mycursor.rowcount, "record inserted.")

In [ ]:
mycursor.execute("SELECT * FROM information_schema.COLUMNS  WHERE TABLE_SCHEMA='data'")

In [ ]:
myresult = mycursor.fetchone()
print(myresult)

In [ ]:
created_at date time timezone user_id username name place tweet mentions urls replies_count 
retweets_count likes_count hastags cashtags link retweet video near geo source user_rt_id user_rt retweet_id
retweet_date translate trans_src trans_dest relevant

In [ ]:
print(data[0]['created_at'], data[0]['date'], data[0]['time'], data[0]['timezone'], data[0]['user_id'], data[0]['username'],
      data[0]['name'], data[0]['place'], data[0]['tweet'], str(data[0]['mentions']), str(data[0]['urls']), data[0]['replies_count'],
      data[0]['retweets_count'], data[0]['likes_count'], str(data[0]['hashtags']), str(data[0]['cashtags']), data[0]['link'], data[0]['retweet'],
      data[0]['video'], data[0]['near'], data[0]['geo'], data[0]['source'], data[0]['user_rt_id'], data[0]['user_rt'],
      data[0]['retweet_id'], data[0]['retweet_date'], data[0]['translate'], data[0]['trans_src'], data[0]['trans_dest'],0,str(data[0]['photos']), 
       str(data[0]['reply_to']))

In [ ]:
import datetime
x = datetime.time(17,59,51)
print(x.date())
t = datetime.time(1, 2, 3)
print(t)
x = datetime.datetime(2020,1,1,17,2,3)
print(x.date().isoformat())
import time    
time.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# !pip install bert-serving-server
# !pip install bert-serving-client

In [ ]:
import nltk

In [ ]:
sentence = "Wow! This needs to be the logo for CVE-2019-11253 how do we get that done"
tokens = nltk.word_tokenize(sentence)

In [ ]:
tagged = nltk.pos_tag(tokens)

In [ ]:
# nltk.download()

In [ ]:
# !python -m nltk.downloader all

In [ ]:
install -c anaconda nltk

In [13]:
import string
array = "maduiersnfotvbyl"
final = ""
for c in "giants":
    for w in string.ascii_lowercase:
       index = ord(w) & 0xF
       lookup = array[index]
       if lookup == c:
           final += w
           break
print("Phase 5 password: " + final)

Phase 5 password: dahkg
